In [7]:
import datetime
import numpy as np
import random
import math
from collections import Counter

#генерация из равномерного распределения
a = 1
b = 5
c = random.uniform(a, b)
print(c)

4.64443109201061


In [5]:
#псевдослучайный генератор
def gen_random_int(first, second):
  date_now = str(datetime.datetime.now())[-6:]
  integer_for_gen = int(date_now + date_now)
  generated = 0
  while(True):
    if generated >= first and generated <= second:
        break
    elif first == second:
        return first
    integer_for_gen /= 17
    generated = integer_for_gen
  return generated

print(gen_random_int(2, 100))

7.974440335770102


In [ ]:
#распределение Коши
x_0 = 0
gamma = 1
c = random.uniform(0, 1)
f = x_0 + gamma * np.tan(np.pi * (c-0.5))
print(f)

Домашнее задание: изучить стандарт NIST проверки псевдослучайных чисел, реализовать как можно больше тестов и проверить ими генератор Python.
Каждый тест выдаёт p-value - вероятность того, что полученный результат мог возникнуть при истинной случайности.

In [8]:
#Frequency (Monobit) Test (встречаются ли 0 и 1 примерно одинаково часто)
def frequency_monobit_test(bits: str):
  n = len(bits)
  s = sum(1 if b == '1' else -1 for b in bits)
  s_obs = abs(s) / math.sqrt(n)
  p_value = math.erfc(s_obs / math.sqrt(2))
  return p_value


#Block Frequency Test (проверяет равномерность внутри блоков)
def block_frequency_test(bits: str, block_size=128):
  n = len(bits)
  num_blocks = n // block_size
  if num_blocks == 0:
    return 0.0
  chi_sq = 0.0
  for i in range(num_blocks):
    block = bits[i*block_size:(i+1)*block_size]
    pi = block.count('1') / block_size
    chi_sq += 4 * block_size * (pi - 0.5) ** 2
  p_value = math.erfc(math.sqrt(chi_sq / (2 * num_blocks)))
  return p_value


#Runs Test (проверяет, часто ли 0 и 1 меняются местами)
def runs_test(bits: str):
  n = len(bits)
  pi = bits.count('1') / n
  if abs(pi - 0.5) > (2 / math.sqrt(n)):
    return 0.0
  Vn = 1
  for i in range(1, n):
    if bits[i] != bits[i - 1]:
      Vn += 1
  p_value = math.erfc(abs(Vn - 2*n*pi*(1-pi)) / (2*math.sqrt(2*n)*pi*(1-pi)))
  return p_value


#Longest Run of Ones in a Block (проверяет, не слишком ли длинные серии подряд идущих 1)
def longest_run_of_ones_test(bits: str, block_size=128):
  n = len(bits)
  num_blocks = n // block_size
  if num_blocks == 0:
    return 0.0
  max_runs = []
  for i in range(num_blocks):
    block = bits[i*block_size:(i+1)*block_size]
    longest = max(len(run) for run in block.split('0'))
    max_runs.append(longest)
  mean_run = np.mean(max_runs)
  return math.exp(-abs(mean_run - 10) / 5)


# Cumulative Sums Test (проверяет, не уходит ли последовательность в одну сторону)
def cumulative_sums_test(bits: str):
  X = [1 if b == '1' else -1 for b in bits]
  S = np.cumsum(X)
  z = max(abs(S))
  n = len(bits)
  p_value = 1 - sum(math.exp(-((4*k+1)**2 * z**2)/(2*n)) for k in range(-10,10))
  return p_value


def generate_bits(n=10000):
  return ''.join(str(random.randint(0, 1)) for _ in range(n))


def run_all_simple(bits: str):
  results = {}
  results["Monobit"] = frequency_monobit_test(bits)
  results["Block Frequency"] = block_frequency_test(bits)
  results["Runs"] = runs_test(bits)
  results["Longest Run of Ones"] = longest_run_of_ones_test(bits)
  results["Cusums"] = cumulative_sums_test(bits)
  return results


if __name__ == "__main__":
  bits = generate_bits(10000)  # генерим последовательность
  results = run_all_simple(bits)

  for test, pval in results.items():
    print(f"{test}: p-value = {pval:.4f} {'(OK)' if pval >= 0.01 else '(FAIL)'}")


Monobit: p-value = 0.9840 (OK)
Block Frequency: p-value = 0.2688 (OK)
Runs: p-value = 0.6171 (OK)
Longest Run of Ones: p-value = 0.4815 (OK)
Cusums: p-value = 0.4100 (OK)


Вывод: видим, что генератор Python работает неплохо.